# Vector Store

In [49]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
load_dotenv()
google_embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
model  = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')




In [50]:
import time
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key='pcsk_6xp3m6_RZNgC8S9uU9hYcS48M8v6wF3pJbeQoRd37kmKtSBb7aNEwgfLn4dKGGEYAjTVDC')
index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [51]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=google_embedding)

In [52]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['0b1feb52-3c9f-48b9-ab91-0a0de5edc2d5',
 '03d80aba-561a-4e39-a8c4-da65e1a427ae',
 'f8159b88-257d-485d-b42a-5b2aafaa4f7a',
 'da1c76a6-18b1-4fc5-83e7-068c2f84665c',
 'cd798676-fdbb-4ea0-84df-f5e7465c2ef9',
 '057b41ba-473a-4e1b-b6c9-d800a28a3bb5',
 'd8a564bb-6df4-41e0-b68f-abe7c0b74675',
 '148fbd3c-bc37-4a13-9d9e-0a7f52e4c600',
 '26e124a1-7df0-4849-9527-5b98b3a12a67',
 '64f1e1dc-ee38-479a-b32f-961cd7e89760']

In [53]:
results = vector_store.similarity_search(
    "What is weatherr forecast for tommorow",
    k=2,
   
)
print(results)

[]


In [57]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('../langchain-tutorial/documents/ragdocs.pdf')
docs = loader.load()



In [58]:
vector_store.add_documents(documents=docs)

['096ba100-e2de-4af2-b324-67c4f55ec05f',
 '6b2a37ab-b9ae-4922-88c7-05d9dc7d9c08',
 'e4c791d1-aefb-4eef-ac1f-6ba58733a8e4']

In [59]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
template = PromptTemplate(
    template='anser the give questtion {question} , based on provided information->{info}',
    input_variables=['question','info']
)

chain = template|model|parser
txt = "who are from charusat universsity"
results = vector_store.similarity_search(
    txt,
    k=3,
   
)

chain.invoke({'question':txt ,'info':results})

'Based on the provided text, Vatsal and Aneri Desai are from Charusat University.'